# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125357e+02     1.611387e+00
 * time: 0.7429399490356445
     1     1.094486e+01     9.075695e-01
 * time: 2.1148128509521484
     2    -1.085422e+01     1.069791e+00
 * time: 2.777911901473999
     3    -3.366967e+01     8.214980e-01
 * time: 3.7973439693450928
     4    -4.709107e+01     5.911101e-01
 * time: 4.772605895996094
     5    -5.660173e+01     2.024942e-01
 * time: 5.756419897079468
     6    -5.958791e+01     1.532272e-01
 * time: 6.444835901260376
     7    -6.080563e+01     5.564726e-02
 * time: 7.124085903167725
     8    -6.131360e+01     4.138753e-02
 * time: 7.7837889194488525
     9    -6.159644e+01     5.025457e-02
 * time: 8.439301013946533
    10    -6.180308e+01     2.824506e-02
 * time: 9.105658054351807
    11    -6.195093e+01     2.425895e-02
 * time: 9.77595591545105
    12    -6.203983e+01     1.966861e-02
 * time: 10.425843000411987
    13    -6.209309e+01     1.618346e-02
 * time: 11.07384204864502
   

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671063
    AtomicLocal         -18.8557628
    AtomicNonlocal      14.8522610
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485357 
    Xc                  -19.3336815

    total               -62.261666461556